In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import sklearn
import matplotlib.pylab as plt

from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix, classification_report

# Experiments:

- **1st experiment**: OASIS-3 data with only normalization and automated crop (no registration, no brain extraction, no augmentation);

- **2nd experiment**: OASIS-3 data with normalization, automated crop and registration w.r.t. AD/CN avg template (no brain extraction, no augmentation);

- **3rd experiment**: OASIS-3 data with normalization, automated crop, registration w.r.t. AD/CN avg template and brain extraction (no augmentation);

- **4th experiment**: OASIS-3 data with normalization, automated crop, registration w.r.t. AD/CN avg template and brain extraction (augmentation is here performed on the training set);

- **Tuned best experiment (4th)**: as follows.

In [ ]:
path = "..." # insert your own path of OASIS-3 data with normalization, automated crop, registration w.r.t. AD/CN avg template and brain extraction applied (augmentation is then performed on the training set)

X = [] 
Y = [] 

dir = path 
classes = ["AD","CN"] 
classes_list = os.listdir(dir) 
classes_list.sort() 

full_path_AD = os.path.join(dir, classes_list[0]) 
samples_AD = os.listdir(full_path_AD) 
samples_AD.sort() 
for a in samples_AD: 
  b = os.path.join(full_path_AD, a) 
  data = np.load(b)
  data = data.astype('float32') 
  temp = [] 
  for i in range(85,135): 
    temp.append(cv2.resize(data[:,:,i], (192,147))) 
  X.append(temp) 
  y = [0]*len(classes_list) 
  y[1] = 1 
  Y.append(y) 

full_path_CN = os.path.join(dir, classes_list[1]) 
samples_CN = os.listdir(full_path_CN) 
samples_CN.sort()
for a in samples_CN: 
  b = os.path.join(full_path_CN, a)
  data = np.load(b)
  data = data.astype('float32')
  temp = []
  for i in range(85,135): 
    temp.append(cv2.resize(data[:,:,i], (192,147)))
  X.append(temp)
  y = [0]*len(classes_list)
  y[0] = 1 
  Y.append(y) 
  
X = np.asarray(X) 
X = X[...,np.newaxis] 
Y = np.asarray(Y) 

print(X.shape)
print(Y.shape)

In [ ]:
model = models.Sequential()
model.add(layers.ConvLSTM2D(filters=8, 
                            kernel_size=(3,3), 
                            return_sequences=False, 
                            data_format="channels_last", 
                            input_shape=(50, 147, 192, 1)))
model.add(layers.Dropout(0.6)) 
model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.6)) 
model.add(layers.Dense(2, activation="softmax"))

model.summary()

opt = tf.keras.optimizers.SGD(learning_rate=0.005) 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [ ]:
seed = 0
n_splits = 5
batch_size = 6

cv = StratifiedShuffleSplit(n_splits=n_splits, train_size=0.8, random_state=seed) 

scores = []
loss_per_split = []
acc_per_split = []
tprs = []
aucs = []
senss = np.zeros(shape=(n_splits))
specs = np.zeros(shape=(n_splits))
f1_scores = np.zeros(shape=(n_splits))
mean_fpr = np.linspace(0,1,100)
plt.figure(num=1, figsize=(10,10))

i = 1
for train, test in cv.split(X, Y):

  X_train, X_val, Y_train, Y_val = train_test_split(X[train], Y[train], test_size=0.2,
                                                  shuffle=True, random_state=seed)
  
  dim = (50, 147, 192)
  X_train_aug = np.empty((352, *dim, 1)) 
  Y_train_aug = np.empty((352, 2))

  for j in range (0, 176): 
    X_train_aug [j*2,:,:,:,:] = X_train[j,:,:,:,:]
    X_train_aug [(j*2)+1,:,:,:,:] = np.flip(X_train[j,:,:,:,:], 1) 
    Y_train_aug [j*2,:] = Y_train[j,:]
    Y_train_aug [(j*2)+1,:] = Y_train[j,:]

  print('-------------------------------------------------------------------------------------------------------------------------------')

  print(f'Training and validation for split {i}:')
  earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1, restore_best_weights=True)
  callbacks = [earlystop]
  history = model.fit(x=X_train_aug, y=Y_train_aug, validation_data=(X_val,Y_val), epochs=50, batch_size=batch_size, verbose=1, callbacks=callbacks) 
  
  print("Evaluation on test data...")
  evaluation = model.evaluate(X[test], Y[test])
  scores.append(evaluation)
  print(f'Scores for split {i}: {model.metrics_names[0]} of {evaluation[0]}; {model.metrics_names[1]} of {evaluation[1]*100}%')
  loss_per_split.append(evaluation[0]) 
  acc_per_split.append(evaluation[1] * 100)

  print('Prediction on test data...')
  pred = model.predict(X[test])

  y_true = Y[test].argmax(axis=-1)
  y_pred = pred.argmax(axis=-1)
  report = classification_report(y_true, y_pred, target_names=['CN','AD'], output_dict=True)
  senss[i - 1] = report['AD']['recall']
  specs[i - 1] = report['CN']['recall']
  f1_scores[i - 1] = report['AD']['f1-score']
  print(classification_report(y_true, y_pred, target_names=['CN','AD']))
  print('Confusion matrix for split ' + str(i) + ':')
  print(confusion_matrix(y_true, y_pred))

  fpr, tpr, thresholds = roc_curve(y_true, pred[:,1], pos_label=1)
  tprs.append(np.interp(mean_fpr, fpr, tpr))
  roc_auc = auc(fpr, tpr)
  aucs.append(roc_auc)
  plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC of split %d (AUC = %0.4f)' % (i, roc_auc))

  i += 1

print('%%%%%%%%% Summary of results on test data %%%%%%%%%')

np_scores = np.array(scores)
losses = np_scores[:, 0:1]
accuracies = np_scores[:, 1:2]
print('Losses:')
print(losses)
print('Accuracies:')
print(accuracies)
print('Sensitivities:')
print(senss)
print('Specificities:')
print(specs)
print('F1-scores:')
print(f1_scores)
print("Avg loss: {0} +/- {1}".format(np.mean(losses), np.std(losses)))
print("Avg accuracy: {0} +/- {1}".format(np.mean(accuracies), np.std(accuracies)))
print("Avg sensitivity: {0} +/- {1}".format(np.mean(senss), np.std(senss)))
print("Avg specificity: {0} +/- {1}".format(np.mean(specs), np.std(specs)))
print("Avg f1-score: {0} +/- {1}".format(np.mean(f1_scores), np.std(f1_scores)))

plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b', label='Mean ROC (AUC = %0.4f $\pm$ %0.4f)' % (mean_auc, std_auc), lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False positive rate (1-specificity)', fontsize=18)
plt.ylabel('True positive rate (sensitivity)', fontsize=18)
plt.title('ROC', fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})

plt.show()

model.save(".../model-best.h5") # choose your own path to store the model weights that led to the lowest validation loss, to be used to evaluate the model performance on independent ADNI-2 data